# Classification of RNA-binding protein binding sites
## Lecture 'ML methods for biological sequence classification'
### Annalisa Marsico and Ernesto Elorduy, TUM 29.01.2020

The goal of this tutorial is to help you familiarize yourself with the use of **S**upport **V**ector **M**achines (SVMs) for classification tasks. Especially the second exercise is meant to show you how SVMs can be used in combination with string kernels to classify biological RNA sequences corresponding to RNA-binding protein sites.

Metazoan genomes encode hundreds of **R**NA-**b**inding **p**roteins (RBPs). These proteins regulate post-transcriptional gene expression and have critical roles in numerous cellular processes including mRNA splicing, export, stability and translation. Despite their ubiquity and importance, the binding preferences for most RBPs are not well characterized. 

High-throughput techniques, such as CLIP-seq experiments (cross-linking immunoprecipitation followed by high-throughput sequencing) allow the genome-wide characterization of RNA-binding protein sites. CLIP-seq experiments employ *’peak-calling’* methods to identify clusters of read coverage that are significantly enriched above background. These clusters can then be classified as *’binding sites’*. This raises some questions: 
- *Can RNA-binding protein sites be discriminated from other RNA regions (or background transcripts) based on sequence alone?* 
- *Which sequence features are enriched in such binding sites for a particular protein under study?* 

In this tutorial we provide fasta sequences of the putative binding sites for two proteins: IGF2BP123 and PUM2. These RBPs are known to be involved in RNA regulation and bind to mRNAs as well as long non-coding RNAs.

The original data was been downloaded from [this database](http://dorina.mdc-berlin.de), converted to fasta files and pre-processed in our lab. Negative/background regions were generated by taking random genomic regions corresponding to transcribed RNAs, maintaining the same length distribution as the binding site sequences. Links for downloading these data sets are provided below.

We will split this classification into the following subtasks:
1. Loading the data
2. Calculating the spectra
3. Doing a train and test split of the sequences
4. Performing a grid search to find good choices for $\mathrm{C}$ and $\mathrm{k}$
5. Extracting the most important k-mers

Install Biopython

In [0]:
!pip install biopython

Import modules and set matplotlib's backend

In [0]:
import Bio.SeqIO as sio
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import precision_recall_curve, precision_score
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as plt
import seaborn
import itertools
import operator
%matplotlib inline

got_time = False

Download the data

In [0]:
# We prefix the "!" character to run bash commands in IPython
!wget -O positive_IGF2BP123.fasta -nv https://hmgubox.helmholtz-muenchen.de/f/845665a7cdd4495aa7db/?dl=1
!wget -O negative_IGF2BP123.fasta -nv https://hmgubox.helmholtz-muenchen.de/f/3947aa537280430bb15b/?dl=1
!wget -O positive_PUM2.fasta -nv https://hmgubox.helmholtz-muenchen.de/f/99a7578b298545da86f5/?dl=1
!wget -O negative_PUM2.fasta -nv https://hmgubox.helmholtz-muenchen.de/f/2b82b3e15ac24197ad84/?dl=1

## 1. Loading the data
_Biopython_ makes loading fasta sequences easy.
Here we use `sio.parse()` on our fasta file. It returns an iterable `SeqRecord` object which we can itarate over, e. g. with a for loop or a comprehension, to construct lists of sequences. 
 

In [0]:
# Choose one RBP (IGF2BP123 or PUM2) and create a list of sequences for each of 
# the positive and negative data sets.
pos_seqs = # your code here 
neg_seqs = # your code here 

Have a look at one of the sequences. Notice that binding site sequences are shown in upper case in the fasta ﬁle and that  sequences ﬂanking the binding sites are displayed in lower case. 

- Are flanking sequences crucial for our classiﬁcation task?

In [0]:
# Have a look at the ouput of sio.parse()
print(type(pos_seqs[0]))
print(pos_seqs[0].seq)

## 2. Calculating the spectrum of each sequence
The general idea is to find a solution that maps a sequence to a vector. In a standard **spectrum kernel**, the vector contains a dimension for each k-mer and therefore:

$\begin{equation}
\phi(x_i) \in \mathbb{R}^{4^k}
\end{equation}$

From an applied point of view, this implies that reasonable choices for $\mathrm{k}$ should not be too large as the vectors are very sparse and can explode in size quickly.

- Try to understand the code below and how the spectrum kernel has been implemented.

- *For the ambitious of you: have a look at [sparse vectors](https://docs.scipy.org/doc/scipy/reference/sparse.html) and see if that you can improve performance or memory. However, this is not mandatory.* (\*)

In [0]:
def get_numbers_for_sequence(sequence):
    result = []
    for letter in sequence:
        if letter == 'A':
            result.append(0)
        elif letter == 'C':
            result.append(1)
        elif letter == 'G':
            result.append(2)
        elif letter == 'T' or letter == 'U':
            result.append(3)
        else:
            return [-1]
    return result

def _extract_spectrum_sequence(sequence, k):
    """Compute k-spectrum for a given sequence and k-mer length k.

    This method computes the spectrum for a given sequence and k-mer-length k.
    The idea is to first create a vector of the given size (4**k) and then
    transform each k-mer to a sequence of length k of numbers 0-3
    (0 = A, 1 = C, 2 = G, 3 = U).
    From there, we can multiply that sequence with a vector of length k,
    containing the exponents of 4 to calculate the position in the spectrum.
    Example: AUUC -> 0331 -> 4**0*1 + 4**1*3 + 4**2*3 + 4**3*0
    """
    n = len(sequence)
    spectrum = np.zeros(np.power(4, k))
    multiplier = np.power(4, range(k))[::-1]
    for pos in range(n - k + 1):
        pos_in_spectrum = np.sum(multiplier * get_numbers_for_sequence(sequence[pos:pos+k]))
        spectrum[pos_in_spectrum] += 1
    return spectrum

def extract_spectrum(sequences, k, include_flanking=False):
    """Compute k-spectra for a set of sequences using k-mer length k.
    
    Computes the k-spectra for a set of sequences. This is done such that the
    resulting vectors can be fed into a linear SVM or other classification
    algorithm. The k-spectrum of a sequence is a sparse vector containing the
    number of times that a k-mer occurs in the given sequence. It has
    as many dimensions as there are possible k-mers.
    
    Parameters:
    ----------
    sequences:              A list of Biopython sequences
    k:                      Integer. The length of kmers to consider
    include_flanking:       Include flanking regions? (the lower-case letters
                            in the sequences given)
    
    Returns:
    -------
    A numpy array of shape (N, 4**k), containing the k-spectrum for each
    sequence. N is the number of sequences and k the length of k-mers considered.
    """
    spectrum = []
    for seq in sequences:
        if include_flanking:
            seq = seq.upper()
        else:
            seq = [x for x in seq if 'A' <= x <= 'Z']
        spectrum.append(_extract_spectrum_sequence(seq, k))
    return np.array(spectrum)

class SpectrumSVM:
    """
    This builds a wrapper for the spectrum calculation and SVM classifier
    to support tuning the parameters via grid search.
    """
    def __init__(self, C=1, k=3, include_flanking=False):
        self.C = C
        self.k = k
        self.include_flanking = include_flanking
        self.clf = SVC(C=self.C, probability=True, kernel='linear')
    
    def fit(self, X, y):
        spectrum_X = extract_spectrum(X, self.k, self.include_flanking)
        self.clf.fit(spectrum_X, y)
    
    def predict_proba(self, X):
        spectrum_X = extract_spectrum(X, self.k, self.include_flanking)
        return self.clf.predict_proba(spectrum_X)

    def predict(self, X):
        spectrum_X = extract_spectrum(X, self.k, self.include_flanking)
        return self.clf.predict(spectrum_X)

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

    def get_params(self,deep=True):
        return {"C" : self.C, "k" : self.k,"include_flanking" : self.include_flanking}

### Test if spectra are correct and plot which k-mers are over-represented

In [0]:
import time
k = 3
start = time.time()
spectrum_pos = extract_spectrum(pos_seqs, k, include_flanking=False)
spectrum_neg = extract_spectrum(neg_seqs, k, include_flanking=False)
print ("Calculated {}-spectrum in {} seconds".format(k, time.time() - start))

list_of_kmers = map(''.join, itertools.product('ACGU', repeat=k))
fig = plt.figure(figsize=(14, 8))
ind = range(4**k)
ax = plt.barh(ind, np.mean(spectrum_pos, axis=0) - np.mean(spectrum_neg, axis=0))
y = plt.yticks(ind, list_of_kmers, fontsize=6)

## 3. Perform a Training and Test split
[scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html?highlight=train%20test#sklearn.model_selection.train_test_split) provides us with the function `train_test_split()` to divide our data into random train and test subsets. This is necessary to see how our model generalizes on previously 'unseen' data.

- Create a train-test split of the feature and target variables. 
- Use a test size of 0.1, and a random state of 42 to ensure reproducibility.

Be sure to do a proper [stratification](https://stats.stackexchange.com/questions/49540/understanding-stratified-cross-validation) of your data. This will help us ensure that our random sample is well balanced (same/similar distribution of classes in both the train and test sets), which reduces estimation errors. Remember that with a simple random split, we may end with a non-representative distribution of our target classes in the test set. 

*Even if we are presented with a balanced dataset, stratification won't harm. It is a good practice to apply stratification on all our train-test splits.*

In [0]:
# Create a train-test split of the feature and target variables. 
# Use a test size of 0.1, and a random state of 42 for reproducibility.
# Hint: start by creating the feature and target variables by concatenating 
# class labes and the spectra created before.

y = # your code here
X = # your code here

X_train, X_test, y_train, y_test = # your code here

print (X_train.shape, X_test.shape)
print (y_train.shape, y_test.shape)

## 4. Hyperparameter Tuning
In order to extract good hyperparameters, a grid search trying out all combinations of parameters in some defined ranges can be performed. For this, [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) provides the function `GridSearchCV()` which optimizes hyperparameters of the estimator by searching through a custom hyperparameter grid for the best cross validation score.

We already performed this tuning and found following parameter candidates: `{'k': 5, 'C': 0.1, 'include_flanking': False}`.

- Intantiate a support vector classifier and fit a model based on the previously split training data.
- *For the ambitious of you: try tuning the hyperparameters $\mathrm{k}$, $\mathrm{C}$, and $\mathrm{include\_flanking}$ by yourself. Have a look at [this example](https://hmgubox.helmholtz-muenchen.de/f/b1d7b638edc2420ea3cb/?dl=1). **Warning:** depending on the number of CV runs as well as the size of the parameter grid selected,* `GridSearchCV()` *might take very long computation times. Alternatively,* `RandomizedSearchCV()` *can be used to save computation resources. This is not mandatory.* (\*)

In [0]:
# best params are {'k': 5, 'C': 0.1, 'include_flanking': False}
start = time.time()
clf = SVC(C=0.1, kernel='linear', probability=True)
clf.fit(X_train, y_train)
print ("Trained linear SVM on {}-spectrum in {} seconds".format(k, time.time() - start))

### Performance Evaluation
Let's have a look at how well we are doing on the test data. Accuracy is a valid metric as we are dealing with a balanced data set, but let's also look at ROC and PR curves.

In [0]:
# how well are we doing on the test set
y_score = clf.predict_proba(X_test)
roc_auc = roc_auc_score(y_score=y_score[:,1], y_true=y_test)
tpr, fpr, _ = roc_curve(y_score=y_score[:,1], y_true=y_test)
fig = plt.figure(figsize=(14, 8))
plt.plot(tpr, fpr, label='Performance Spectrum Kernel (AUC: {:0.2f})'.format(roc_auc),
         lw=4, color='orange')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.plot([0, 1], [0, 1], color='navy', lw=4, linestyle='--', label='random')
plt.legend(loc='lower right')
print ("Test Set ROC-AUC: {}".format(roc_auc))

In [0]:
pr_auc = average_precision_score(y_score=y_score[:,1], y_true=y_test)
pr, rc, _ = precision_recall_curve(y_test, y_score[:,1])
fig = plt.figure(figsize=(14, 8))
plt.plot(rc, pr, label='Performance Spectrum Kernel (AUC: {:0.2f})'.format(pr_auc),
         lw=4, color='orange')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.plot([0, 1], [0.5, 0.5], color='navy', lw=4, linestyle='--', label='Random')
plt.ylim([0, 1.05])
plt.legend(loc='upper right')
print ("Test Set PR-AUC: {}".format(pr_auc))

## 5. Extracting the most important k-mers
Now that we have optimal parameters and are sure that our algorithm does reasonably well on the test data, let's have a look at the most relevant k-mers. 

- Do they make sense from a biological point of view?
  - Search the literature for known information about the binding motif of the analyzed RBP.

- Which features (k-mers) contribute the most to discriminate real binding sites from background regions?
  - Find the important features by extracting the weights.

In [0]:
list_of_kmers = map(''.join, itertools.product('ACGT', repeat=k))
coefs_sorted, kmers_sorted = zip(*sorted(zip(clf.coef_.reshape(-1), list_of_kmers),
  key=operator.itemgetter(0), reverse=True))

max_kmers = 50
fig = plt.figure(figsize=(20, 10))
ind = range(min(clf.coef_.shape[1], max_kmers))
ax = plt.bar(ind, coefs_sorted[:max_kmers])
x = plt.xticks(ind, kmers_sorted[:max_kmers], rotation=45)